In [1]:
from __future__ import absolute_import
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D, Dropout
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam, RMSprop
from keras.losses import binary_crossentropy
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy.random as rng
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import random
import time
from tqdm import tqdm
%matplotlib inline

Using Theano backend.


In [2]:
def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(input_dim,), activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    return seq


def create_conv_network(input_dim):
    convnet = Sequential()
    convnet.add(Conv2D(64,(10,10),activation='relu',input_shape=input_dim,
                       kernel_initializer=W_init,kernel_regularizer=l2(2e-4)))
    convnet.add(MaxPooling2D())
    convnet.add(Conv2D(128,(7,7),activation='relu',
                       kernel_regularizer=l2(2e-4),kernel_initializer=W_init,bias_initializer=b_init))
    convnet.add(MaxPooling2D())
    convnet.add(Conv2D(128,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
    convnet.add(MaxPooling2D())
    convnet.add(Conv2D(256,(4,4),activation='relu',kernel_initializer=W_init,kernel_regularizer=l2(2e-4),bias_initializer=b_init))
    convnet.add(Flatten())
    convnet.add(Dense(4096,activation="sigmoid",kernel_regularizer=l2(1e-3),kernel_initializer=W_init,bias_initializer=b_init))
    
    return convnet

In [3]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) +
                  (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))



def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(input_dim,), activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    return seq


def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))


input_dim = 400

# network definition
base_network = create_base_network(input_dim)

input_a = Input(shape=(input_dim,))
input_b = Input(shape=(input_dim,))

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

prediction = Dense(1,activation='sigmoid')(distance)

model = Model(inputs=[input_a, input_b], outputs=prediction)

# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 400)           0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 400)           0                                            
____________________________________________________________________________________________________
sequential_1 (Sequential)        (None, 128)           84352                                        
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 1)             0                                            
___________________________________________________________________________________________

# LOAD DATA

In [4]:
print ('Loading data...')
train_df = pd.read_pickle('train_shuffled.h5').sample(frac=0.25)
# train_df = pd.read_pickle('train_shuffled+duplicates.h5')
test_df  = pd.read_pickle('test.h5').sample(frac=0.01)
print ('Sucessfully load...')

Loading data...
Sucessfully load...


In [5]:
# print ('Processing train and test data')
# x_left_train  = train_df.vec1.values
# x_right_train = train_df.vec2.values
# y_train       = train_df.is_duplicate.values
# train_df = None
# x_left_train = np.array([np.array(row) for row in x_left_train])
# x_right_train = np.array([np.array(row) for row in x_right_train])
    
# x_left_test  = test_df.vec1.values
# x_right_test = test_df.vec2.values
# y_test       = test_df.is_duplicate.values
# test_df = None
# x_left_test = np.array([np.array(row) for row in x_left_test])
# x_right_test = np.array([np.array(row) for row in x_right_test])
# print ('Done.')

Processing train and test data


In [8]:
# callbacks = [ModelCheckpoint(save_best_weights, monitor='val_loss', save_best_only=True),
#              EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto')]

# t0 = time.time()

# history = model.fit([x_left_train, x_right_train], y_train,
#                        batch_size=200,
#                        epochs=100,
#                        validation_data=([x_left_test, x_right_test], y_test),
#                        verbose=1,
#                        shuffle=True,
#                        callbacks = callbacks)

# t1 = time.time()
# print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Train on 99331 samples, validate on 9073 samples
Epoch 1/5
99331/99331 [==============================] - 2s - loss: 0.2501 - accuracy: 0.5164 - val_loss: 0.2284 - val_accuracy: 0.9927

# TODO BATCH LEARNING

In [6]:
def generate_batch(x_left, x_right, y, size):
    while True:
        for i in xrange(0, len(x_left), size):
            x_left_b   = x_left[i:i+size]
            x_right_b  = x_right[i:i+size]
            y_b        = y[i:i+size]

            x_left_b   = np.array([np.array(row) for row in x_left_b])
            x_right_b  = np.array([np.array(row) for row in x_right_b])

            yield ([x_left_b, x_right_b], y_b)

In [11]:
x_left = train_df.vec1.values[:10000]
x_right = train_df.vec2.values[:10000]
y_train = train_df.is_duplicate.values[:10000]

x_left_test  = test_df.vec1.values[:8000]
x_right_test = test_df.vec2.values[:8000]
y_test       = test_df.is_duplicate.values[:8000]

batch_s = 200
batch_total = int(round((1.0*len(x_left))/batch_s))
val_steps = int(round((1.0*len(x_left_test))/batch_s))

callbacks = [ModelCheckpoint('second_try.h5', monitor='val_loss', save_best_only=True),
             EarlyStopping(monitor='val_loss', patience=30, verbose=1, mode='auto')]

t0 = time.time()



history = model.fit_generator(generate_batch(x_left, x_right, y_train, batch_s),
                                samples_per_epoch=batch_total, 
                                nb_epoch=10,
                                verbose=1,
            #                     shuffle=True,
                                validation_data=generate_batch(x_left_test, x_right_test, y_test, batch_s),
                                validation_steps=val_steps,
                                callbacks = callbacks
                               )

t1 = time.time()
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Epoch 1/10
 7/50 [===>..........................] - ETA: 0s - loss: 7.7604e-04 - accuracy: 1.0000

/home/loopdigga/Documents/ml/ml_env/lib/python2.7/site-packages/ipykernel/__main__.py:25: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
/home/loopdigga/Documents/ml/ml_env/lib/python2.7/site-packages/ipykernel/__main__.py:25: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., verbose=1, validation_data=<generator..., steps_per_epoch=50, epochs=10, callbacks=[<keras.ca..., validation_steps=40)`


50/50 [==============================] - 1s - loss: 0.0011 - accuracy: 0.9994 - val_loss: 0.0101 - val_accuracy: 0.9891

In [12]:
summary_stats = pd.DataFrame({'epoch': [ i + 1 for i in history.epoch ],
                              'train_loss': history.history['loss'],
                              'valid_loss': history.history['val_loss']})
summary_stats.to_pickle('history2.h5')

NameError: name 'history' is not defined